In [ ]:
import os
from openai import OpenAI

from dotenv import load_dotenv
load_dotenv("../keys.env")

openai_api_key = os.getenv('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = openai_api_key

In [ ]:
client = OpenAI()

In [ ]:
def generate_question_from_passage(passage, model="gpt-4o"):
    """
    GPT-4에게 단락을 주고, 예상 질문을 생성하도록 요청하는 함수
    """
    prompt = f"아래 단락을 보고 예상 질문을 만들어 주세요.\n\n단락: {passage}\n질문:"
    
    completion = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "너는 단락을 보고 가장 적절한 질문을 생성하는 AI이다."},
            {"role": "user", "content": prompt},
        ]
    )
    
    response = completion.choices[0].message.content
    return response

In [ ]:
def calculate_question_probability(original_question, generated_question, model="gpt-4o"):
    """
    GPT-4에게 유저의 원래 질문이 생성된 질문과 일치할 확률을 계산하게 하는 함수
    """
    prompt = (
        f"유저의 질문: {original_question}\n"
        f"모델이 생성한 질문: {generated_question}\n\n"
        "이 두 질문이 얼마나 유사한지, 유사도를 0에서 1 사이의 숫자로 평가해 주세요."
    )
    
    completion = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "너는 두 질문이 얼마나 유사한지 평가하는 AI이다."},
            {"role": "user", "content": prompt},
        ]
    )
    
    response = completion.choices[0].message.content
    return response

In [ ]:
def upr_rerank(original_question, passages, model="gpt-4o"):
    """
    UPR 리랭커 함수: 문서 패시지를 재정렬하는 메인 함수
    """
    reranked_passages = []
    
    for passage in passages:
        # 문서에서 예상 질문 생성
        generated_question = generate_question_from_passage(passage, model)
        
        # 생성된 질문이 원래 질문과 얼마나 일치하는지 확률 계산
        probability_score = calculate_question_probability(original_question, generated_question, model)
        
        # 점수와 함께 패시지 저장
        reranked_passages.append({
            'passage': passage,
            'generated_question': generated_question,
            'score': probability_score
        })
    
    # 점수에 따라 패시지를 다시 정렬 (내림차순)
    reranked_passages.sort(key=lambda x: x['score'], reverse=True)
    
    return reranked_passages

# 예시
original_question = "걸그룹 에스파의 대표곡이 뭐야?"
passages = [
    "에스파는 2020년에 타이틀곡 Black Mamba로 데뷔한 4인조 걸그룹이다.",
    "에스파 멤버 중 윈터와 카리나는 특유의 케미로 큰 인기를 얻었다.",
    "에스파는 Next Level 곡으로 큰 히트를 거두고, 이 후 Spicy, Drama, Supernova 등의 곡들 역시 큰 인기를 얻었다."
]

# UPR 리랭킹 실행
reranked_results = upr_rerank(original_question, passages)

# 결과 출력
for idx, result in enumerate(reranked_results):
    print(f"순위 {idx + 1}: 점수 {result['score']}, 생성된 질문: {result['generated_question']}, 내용: {result['passage']}")
